In [30]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

def read_one_user_labels(n):
    transport_df = pd.read_csv('../Data/0'+str(n)+'/labels.txt', sep="\t")
    
    directory = '../Data/0'+str(n)+'/Trajectory'
    for filename in os.listdir(directory):
        dt_object = str(datetime.strptime(filename[:-4], '%Y%m%d%H%M%S'))
        if dt_object >= transport_df['Start Time'][0] and \
            dt_object <= transport_df['End Time'][transport_df.shape[0]-1]:
            # Acessa o arquivo .plt e inclui todos os registros com o mesmo
            # identificador (index de transport_df)
            
            # use https://numpy.org/doc/stable/reference/generated/numpy.searchsorted.html
            
            
            
            pass
    return transport_df

transport_df = read_one_user_labels(10)
transport_df

,Start Time,End Time,Transportation Mode
0,2007/06/26 11:32:29,2007/06/26 11:40:29,bus
1,2008/03/28 14:52:54,2008/03/28 15:59:59,train
2,2008/03/28 16:00:00,2008/03/28 22:02:00,train
3,2008/03/29 01:27:50,2008/03/29 15:59:59,train
4,2008/03/29 16:00:00,2008/03/30 15:59:59,train
...,...,...,...
429,2008/12/07 10:30:54,2008/12/07 10:34:14,taxi
430,2008/12/07 10:59:29,2008/12/07 11:29:48,train
431,2008/12/07 11:43:12,2008/12/07 12:23:26,bus
432,2008/12/07 12:23:34,2008/12/07 12:25:07,walk
